--- 

# Learning in Public

* https://twitter.com/David__Colton/status/1591760419714392064?s=20&t=tCW-IPHsCRYaeEZ3uLCGCw
* https://twitter.com/David__Colton/status/1591790353044197376?s=20&t=L3lLOoqrSHJ6GacqwbZNZA
* https://twitter.com/David__Colton/status/1591794596874715136?s=20&t=k1ksVLLScWtOJYF9TCEJfg
* https://twitter.com/David__Colton/status/1592648954457444352?s=20&t=o6ku8KtedaHt0xiH2c5faw
* https://twitter.com/David__Colton/status/1592807007983370240?s=20&t=iczl6WTNuOcxbk7qk2Etrw
* https://twitter.com/David__Colton/status/1594125809278914560?s=20&t=Eb6lYuAe2CTJ3aJ4mMtvhg
* https://twitter.com/David__Colton/status/1594125809278914560?s=20&t=Eb6lYuAe2CTJ3aJ4mMtvhg
* https://twitter.com/David__Colton/status/1594301509109366784?s=20&t=a3tZxY9xjDYCNDY-u5TbSg
* https://twitter.com/David__Colton/status/1594304549128396801?s=20&t=lZUohS_qX5Ln991DOAheKA
* https://twitter.com/David__Colton/status/1594325487119212545?s=20&t=b1pWzraX1UOmy1vqWBRHAw
* https://twitter.com/David__Colton/status/1594612964036050945?s=20&t=LThGDcxBe-Aye148Sq7X7w
* https://twitter.com/David__Colton/status/1594619087551926273?s=20&t=kfZYn3ouCo8FWNENSrpuDg
* https://twitter.com/David__Colton/status/1594622227730911233?s=20&t=idxgHVanIvGTVvXuuGAozQ
* https://twitter.com/David__Colton/status/1594680435753336833?s=20&t=ON7z61vPujVTWJ2y2T_Erwv
* https://twitter.com/David__Colton/status/1594682207930847238?s=20&t=ipaFy4EZNv66x7al50COWA